# Lab 6: Group, Join, Conditionals, Iteration, Randomness. 

Welcome to Lab 6! This week, we will get a bit more practice with grouping and joining tables, using iteration and simulations, and practicing the concept of randomness and probability. This material is covered in [Chapter 9](https://www.inferentialthinking.com/chapters/09/randomness.html) and the prior chapters (e.g., [Chapter 8.4](https://www.inferentialthinking.com/chapters/08/4/Joining_Tables_by_Columns)).

In [ ]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

from client.api.notebook import Notebook
ok = Notebook('lab06.ok')
_ = ok.auth(inline=True)

**Important**: The `ok` tests don't usually tell you that your answer is correct. More often, they help catch careless mistakes. It's up to you to ensure that your answer is correct. If you're not sure, ask someone (not for the answer, but for some guidance about your approach). Basically, for your solution to be correct, it is **necessary** that the tests are passed; however, passing the tests is not **sufficient** for your solution to be correct.

## 1. The Method `group` and Extravaganza Lineup

### Review of the method `group`

Before we work on UCSB's Extravaganza data, let's work on a toy example to review some basics about `group`.

Run the following cell to create a table. This table contains some anonymous ratings (1, 2, 3, 4, or 5 stars) for three flavors of ice creams. (Those flavors can be found at [McConnell's Fine Ice Creams](https://mcconnells.com/)!)

In [ ]:
ratings = Table().with_columns("Flavor", make_array("Banana & Salted Caramel", "Dutchman's Chocolate", 
                                                    "Eureka Lemon & Marionberries", "Banana & Salted Caramel", 
                                                    "Dutchman's Chocolate", "Eureka Lemon & Marionberries", 
                                                    "Banana & Salted Caramel", "Dutchman's Chocolate", 
                                                    "Eureka Lemon & Marionberries", "Banana & Salted Caramel"),
                            "Rating", make_array(2,3,5,1,4,3,4,3,4,5))
ratings

Suppose we want to count how many ratings are collected for each flavor of the ice cream.

In [ ]:
ratings.group("Flavor")

The `group` method with a single argument (the name of a column) counts the number of rows for each category in a column. The result contains one row per unique value in the grouped column. The call to `group` creates a column of counts in each category. The column is called `count` by default, and contains the number of rows in each category.

So in the example table above, we have collected 4 ratings for Banana & Salted Caramel, 3 ratings for Dutchman's Chocolate and 3 ratings for Eureka Lemon & Marionberries.

Now suppose we would like to look at individual ratings for each flavor of the ice cream.

In [ ]:
ratings.group("Flavor", list)

When we use the `group` method with an optional second argument `list`, for each flavor, we have a list of the ratings this particular flavor received. Banana & Salted Caramel received 2, 1, 4 and 5 stars, Dutchman's Chocolate received 3, 4 and 3 stars, and Eureka Lemon & Marionberries received 5, 3 and 4 stars.

There are also other optional second arguments, which will accomplish different objectives. Read the textbook for more!

Now suppose that we want to sort in ascending order the list of ratings for each flavor. We will sort the ratings first and then group them, which will maintain their ratings in sorted order. **The order of these operations is important.**

In [ ]:
ratings.sort("Rating").group("Flavor", list)

Compare the two tables above. The second table now has each list of ratings sorted!

### Extravaganza Data Analysis

Every spring, UCSB hosts Extravaganza, a one-day on-campus music festival. The following questions are based on last year's festival. The AS Program Board (an on-campus organization tasked with organizing entertainment events) sends out a survey to UCSB students asking for their suggestions for music artists. The instructions in the survey specify that each student should select a first choice artist (rank 1), a second choice artist (rank 2), and a third choice artist (rank 3). Run the following cell to see how the first several students responded.

In [ ]:
survey = Table().read_table("survey.csv")
survey

After these responses come in, however, the AS Program Board notices that their survey form does not actually enforce that each participant choose a single first choice artist, a single second choice artist, and a single third choice artist. Run the cell below to see an example of a student who did not follow the survey's instructions. 

In [ ]:
survey.where("Perm Number", are.equal_to(5978341))

The AS Program Board decides to identify all students who did not follow the survey's instructions, delete their votes from the table, and email the students to tell them that their votes did not comply with the rules, and that they should revote if they want to have a say in the Extravaganza lineup. 

The email addreses of all students are available in the student database, a portion of which is displayed in the table below.

In [ ]:
database = Table().read_table("student_data.csv")
database

**Question 1.1**  Use the survey data and the student database information to identify the students who did not follow the survey's instructions. Make an **array** called `violators` that contains the **email addresses** of all students who did not follow the survey's instructions. 

*Hint 1:* Use the `group` command with second argument `list` to see how each student voted.

*Hint 2:* A vote is valid if and only if the participant chose a single first choice artist, a single second choice artist, and a single third choice artist. That means in the table returned by the `group` method, a vote is valid if and only if its rank list is one of `[1, 2, 3]`, `[1, 3, 2]`, ..., `[3, 1, 2]`, i.e., any permutation of `[1, 2, 3]`.

*Hint 3:* However, it's kind of tedious to compare with a vote's rank list with each permutation. Sort the data in some fashion before using the `group` method so that each student who voted correctly has a rank list that looks like `[1, 2, 3]`. Your task now is to find those whose vote does not look like this array.

*Hint 4:* Once you have all the violators' perm numbers, use the `database` table to **get their email addresses**. Remember that you can use the `where` with the predicate `are.contained_in`.

In [ ]:
violators = ...
violators

In [ ]:
_ = ok.grade('q1_1')

**Question 1.2** Now delete the rows from the table `survey` that correspond to the voters in `violators`. After this, the table `survey` should have only valid votes in it.

In [ ]:
survey = ...
survey

In [ ]:
_ = ok.grade('q1_2')

After months of collecting votes and contacting artists, the 2017-18 UC Santa Barbara Extravaganza lineup has officially been released! Run the following cell to see a table of the scheduled performers and their respective numbers of Instagram followers (in thousands).

In [ ]:
extravaganza_performers = ["Dillon Francis", "Charli XCX", "Cardi B", "Coast Modern"]
extravaganza_instagram = [2105, 3101, 35100, 15.5]

extravaganza_lineup = Table().with_columns("Artists", extravaganza_performers, "Instagram", extravaganza_instagram)
extravaganza_lineup.show()

As we come closer to the event, we get word that one of the performers is unable to make it to Extravaganza, and has been replaced by another performer instead! Run the following cell to see a table of the new lineup and their respective numbers of Twitter followers (in thousands).

In [ ]:
new_performers = ["Dillon Francis", "Charli XCX", "DRAM", "Coast Modern"]
new_twitter = [1057, 3249, 123, 8]

new_lineup = Table().with_columns("Performers", new_performers, "Twitter", new_twitter)
new_lineup.show()

**Question 1.3** Use the `join` method to join these two tables together so each row contains the name of the performer, their number of Instagram followers (in thousands), and their number of Twitter followers (in thousands). Save this new table into the variable `lineup_data`.

*Hint:* Here is an [example](https://www.inferentialthinking.com/chapters/08/4/Joining_Tables_by_Columns.html) using `join`.

In [ ]:
lineup_data = ...
lineup_data

In [ ]:
_ = ok.grade('q1_3')

**Question 1.4** You should notice that a couple of artists are missing. Which ones are missing and why are they not in the new table?

Replace this text with your answer

**Question 1.5** Let's add `DRAM` back into the `lineup_data` table so that we can see all the artists who actually performed at Extravaganza last year. DRAM currently has 294.8 thousand Instagram followers and 122.5 thousand Twitter followers.

In [ ]:
lineup_data = ...
lineup_data

In [ ]:
_ = ok.grade('q1_5')

## 2. Getting Hold of Your Friend

You are excited to go to Extravaganza but you don't want to go alone! You are trying to get a hold of your friend to see if they want to go to Extravaganza with you. However, each time you call your friend, the probability that they answer their phone is 1/3. If you call your friend two times today, what is the chance that you will talk to them?

Here is the equation to help you find the probability:

$$P(reaching\ your\ friend\ at\ least\ once\ in\ N\ times) = 1 - P(not\ reaching\ your\ friend\ all\ N\ times)$$

You can find out more about this equation in the textbook [here](https://www.inferentialthinking.com/chapters/09/5/Finding_Probabilities#at-least-one-success) under "At Least One Success" for an example on the probability of rolling a 6 on a die. 

**Question 2.0** Just to make sure that you are on the right track and are able to compute the probability, tell us, what's the probability that you rolled a 6 if you roll a die once?

In [ ]:
prob_rolling_six = ...

In [ ]:
_ = ok.grade('q2_01')

Now, what is the probability that you rolled 6 two times in a row? Well, you need to make sure that you roll a 6 first, followed by another 6. Since each roll is independent from each other, the final probability of rolling a 6 twice in a row is the product of individual probabilities. Compute `prob_rolling_six_twice` using `prob_rolling_six` in your answer below.

In [ ]:
prob_rolling_six_twice = ...

In [ ]:
_ = ok.grade('q2_02')

OK, let's get back to your friend that you want to invite to go to Extravaganza.

**Question 2.1** Let's first calculate the probability that your friend will not answer the phone both times you call them.

In [ ]:
no_answer = ...
no_answer

In [ ]:
_ = ok.grade('q2_1')

**Question 2.2** Now that we have the probability of your friend not answering both times, let's calculate the probability that you will reach your friend at least once out of the two calls (using the formula from above).

In [ ]:
answered = ...
answered

In [ ]:
_ = ok.grade('q2_2')

## 3. Memes

<img src="silicon-meme.jpg" width=40%><img src="reaction-meme.jpg" width=40%>

Twitter has just hired you to analyze some of its most popular memes! Run the following cell to see a table of information on recent Twitter posts that contained memes of certain popular formats. For each Twitter post, the table contains
* The format of the meme in the post. For example, the format *Chemistry Cat* shows a cat dressed up as a scientist in a chemistry lab. The sign above shows a question that is answered below with a witty comment involving a chemical element or a chemistry concept. Two examples of a meme in this format are shown above.
* The Twitter handle (username) of the person who made the post.
* The number of retweets (shares).
* The number of likes.
* The number of days from when the post was generated to when you got the dataset.


In [ ]:
memes = Table.read_table('memes.csv')
memes.show()

**Question 3.1** Twitter is interested in determining which meme formats get the most retweets and likes. Calculate the total number of retweets and likes associated with each of the meme formats, and save a table of these results in a variable called `retweets_likes`. Your table should have three columns, containing, from left to right:
* The format of the meme.
* The total number of retweets for all memes with this format.
* The total number of likes for all memes with this format.

*Hint 1:* You should use the `group` method with an optional second argument called `sum`. Look at the [textbook](https://www.inferentialthinking.com/chapters/08/2/Classifying_by_One_Variable.html) if you are not familiar with this argument. (It might remind you of how we used `apply` method with a function name to apply to the columns.)

*Hint 2:* The table `retweets_likes` should not have the column called `Number of Days Since Post` or `Posted By`. Remove those columns before using the `group` method.

In [ ]:
retweets_likes = ...
retweets_likes

In [ ]:
_ = ok.grade('q3_1')

**Question 3.2** The total number of retweets and likes should be taken relative to the number of days since the meme was posted, because memes that have been posted for longer will naturally have more of a chance to gather retweets and likes. For each meme format, calculate the number of days since a meme of that format was first posted, and add a column called `age` with these results to the table `retweets_likes`, saving your new table in a variable called `retweets_likes_age`.

*Hint:* The number of days since a meme of that format was first posted is the maximum of `Number of Days Since Post` for that particular format. Use the method `group` with a second optional argument called `max`.

In [ ]:
retweets_likes_age = ...
retweets_likes_age

In [ ]:
_ = ok.grade('q3_2')

**Question 3.3** Rank the meme formats by popularity. The popularity of a meme format is measured as the total number of retweets and likes per day since the meme format was originally posted. In other words, we define the popularity of a meme format by the formula below.

\begin{equation*}
\text{popularity} = (\text{retweets} + \text{like})/\text{age}
\end{equation*}

Create an array called `popular_memes` that contains the meme formats ranked by popularity, so that the most popular meme is first in the array, and the least popular meme is last.

*Hint:* We recommend you to create a new table with an additional column called `popularity` and sort the table.

In [ ]:
popular_memes = ...
popular_memes

In [ ]:
_ = ok.grade('q3_3')

## 4. Yahtzee 

In the dice game Yahtzee, players roll and reroll dice, trying to meet certain objectives. A player rolls five dice on the first roll, and after looking at the results, *can choose to* reroll any number of them on the second roll. Similarly, after looking at the results of the second roll, the player can choose to reroll any number of those for the third roll. After the third roll, no more rolling is allowed.

One objective in Yahtzee is to roll as many 6's as possible. The standard strategy is as follows:
* Roll all five dice.
* Keep any that are 6's. Reroll all other dice.
* Keep any that are 6's. Reroll all other dice.
The number of 6's at the end of this process determined the player's score. 

**Question 4.1** Create an array called `my_dice` that contains the results of a first Yahtzee roll (that is, five random numbers between 1 and 6).

In [ ]:
my_dice = ...
my_dice

In [ ]:
my_dice = np.arange(1,7)
my_dice

In [ ]:
_ = ok.grade('q4_1')

**Question 4.2** Define a function called `reroll()` that takes an array as an input parameter (e.g., an array that holds the five dice rolls). This function generates an array that contains the contents of the input array with the results after one additional Yahtzee roll. Your function should implement the standard Yahtzee strategy for rolling 6's, that is, keep all dice that were a 6 and reroll all other dice. Your solution would need to check which dice in the array are not 6 and re-roll only those dice.

*Hint:* You can test out your function by repeatedly rerolling. Since you are keeping all the 6's you ever roll, eventually you should get all 6's by repeatedly rerolling. The dice that were 6 should not change as you reroll.

_Note: as you are rerolling the dice, even if your function is not returning the array, Python will remember any modifications that you made to the **array** that you passed as the input parameter even after you exit from the function._

*Hint:* As you are checking the elements of the array, you _do not_ want to use `for dice in rolled_dice:`, because the variable `dice` would not know _which_ element it is looking at: it will only know the _value_ of each element not its _index_ (e.g., it will know that you are looking at the _value_ 5 but doesn't know if it is in the first or the i-th position in the array). Take a look at the examples of the `for` loop we did in class or in the previous lab to see how to get the _index_ of the value in an array.

In [ ]:
def reroll( rolled_dice ):
    ...

In [ ]:
# You can rerun this cell repeatedly to test your code!
reroll( my_dice )
my_dice

Now, practice taking a complete turn at Yahtzee, and see how many 6's you can get! Re-run the code cell from Question 4.1 to roll new dice. Then use your `reroll( my_dice )` function twice, and calculate the number of 6's you have at the end of your turn. 

In [ ]:
# Practice taking a turn here. How many 6's did you get?


**Question 4.3** Now, use a `for` loop to help you take 100,000 turns at Yahtzee. On each turn, you should roll the dice (all five of them), reroll them twice (rerolling means you only reroll the dice that are not sixes; use the `reroll()` function you already defined), and calculate the number of 6's you have at the end of your turn.

Create an array called `sixes` that contains the number of 6's you had at the end of each turn. This array should have 100,000 entries.

*Hint*: Try taking 10 turns with a `for` loop. Once you are sure you have that figured out, change it to 100,000 turns. It will take a little while (about a minute) for Python to perform the calculations when you are doing 100,000 turns.

In [ ]:
sixes = ...    
sixes

In [ ]:
_ = ok.grade('q4_3')

### Expected Value (EV)

In the next question, we are going to work with a statistical term called _expected value_ (EV). EV is essentially an average, except that EV takes into account the probability of getting each value. Here's an example from the [Statistics How To](https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/expected-value/) website:

**You toss a fair coin three times. $X$ is the number of heads which appear. What is the EV?**

**Step 1**: Figure out the **possible values for $X$**. For a three-coin toss, you could get anywhere from 0 to 3 heads. So your values for X (the number of heads in 3 tosses) are 0,1,2 and 3.

**Step 2**: Figure out your **probability of getting each value of $X$**. 
You may need to use a sample space, which contains all possible outcomes. (The sample space for this problem is: `{HHH TTT TTH THT HTT HHT HTH THH}`). 
The probabilities are: 
* $1/8$ for 0 heads (only tails came up on each coin toss), 
* $3/8$ for one head (3 outcomes contain exactly one head in three tosses), 
* $3/8$ for two heads, and 
* $1/8$ for 3 heads (three heads in three coin flips can happen in exactly one way).

**Step 3**: Multiply your $X$ values in Step 1 by the probabilities from step 2 to get the expected value which is usually written as $E(X)$.

$E(X) = 0\times(1/8) + 1\times(3/8) + 2\times(3/8) + 3\times(1/8) = 3/2$.

The resulting EV is $3/2 = 1.5$.

Thus, if you collected data by repeatedly flipping three coins, you would say that the expected number of heads you see is about 1.5 (which means you would expect 1 or more heads more often).

_Note that the expected value is **not** a probability (its values can be way larger than probability's max value of 1)._



You can read more about how to calculate expected values in the article "_Finding the Expected Value of a Dice Game_" in the Method 3: https://www.wikihow.com/Calculate-an-Expected-Value.


### Expected Value (EV) of a specific outcome

The above example talks about the number of heads in hypothetical coin tosses. How would you compute the expected value of _your specific coin tosses_? 

Imagine your three coin flips resulted in `HTH` (two heads and one tail). In order to compute the EV value, we would count the total number of times the event (`H`) had occurred out of the total number of trials (3 coin tosses), which in this case would give us $2/3$. We are essentially computing _the average value_ we got in this specific experiment. 

Let's get back to our Yahtzee rolls.

**Question 4.4** Use the data you have collected in your array `sixes` to approximate **the number of 6's** you would you expect to get in one turn using this strategy. Store your result in a variable called `expected_sixes.` 

Note that this **does not** need to be a whole number. 


*Hint:* How to calculate expected values based on the array `sixes`? Look at the example above. In this example, if you only took five turns and got 3, 4, 1, 2 and 3 in `sixes`, the expected value of the number of sixes is $(3 + 4 + 1 + 2 + 3) / 5$. Generalize this to the array `sixes` which has 100,000 turns. (_Use the methods that can determine the sum of an array and the length of an array!_)

In [ ]:
expected_sixes = ...
expected_sixes

In [ ]:
_ = ok.grade('q4_4')

**Question 4.5** Use the data you have collected in `expected_sixes` to approximate **the most commonly rolled number of 6's** when taking a single turn using this strategy. Store your result in a variable called `most_common_sixes.`

Note that this **does** need to be **a whole number**, because it is impossible to roll a non-integer number of sixes in a single turn of Yahtzee. You are going to look at the valus you collected in `expected_sixes` and see which value came up most often. 

_Hint: You can count the values or you can also look at the `mode` (used the same way you'd use `sum` and `max`)._

In [ ]:
most_common_sixes = ...
most_common_sixes

In [ ]:
_ = ok.grade('q4_5')

Congratulations, you completed Lab 6!

To submit:

1. Select `Run All` from the `Cell` menu to ensure that you have executed all cells, including the test cells. 
2. **Save and Checkpoint** from the `File` menu,
3. Read through the notebook to make sure everything is fine.
4. Submit using the cell below.

In [ ]:
_ = ok.submit()